In [1]:
import warnings
warnings.filterwarnings("ignore")
from src.dataset import load_tokenized_data
from src.model import LyricsMelodyModel
from src.embeddings import extract_embedding_weights
from tensorflow.python.keras.callbacks import TensorBoard

Using TensorFlow backend.


In [2]:
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=False)                    

In [3]:
x, y, tokenizer, songs = load_tokenized_data(with_melody=True, melody_type='doc2vec')
embedding_matrix = extract_embedding_weights()

100%|█████████████████████████████████████████████████████| 614/614 [00:01<00:00, 364.50it/s]


In [4]:
model = LyricsMelodyModel(tokenizer, embedding_matrix, 
                          rnn_type='GRU', 
                          rnn_units=20, 
                          bidirectional=False,
                          dropout=0.4,
                          train_embedding=True, 
                          is_layer_norm=True)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 300)       2247600     input_1[0][0]                    
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 300)          0           embedding_1[0][0]                
__________________________________________________________________________________________________
input_2 (InputLayer) 

In [5]:
model.train([x,songs], y, epochs=20, batch_size=64, callbacks=[tensorboard])

Instructions for updating:
Use tf.cast instead.
Train on 171249 samples, validate on 19028 samples
Epoch 1/20
171249/171249 [==============================] - 22s 128us/step - loss: 6.5551 - _perplexity: 213.7566 - val_loss: 6.1728 - val__perplexity: 274.7551
Epoch 2/20
171249/171249 [==============================] - 18s 106us/step - loss: 5.6517 - _perplexity: 179.7978 - val_loss: 5.9913 - val__perplexity: 430.8348
Epoch 3/20
171249/171249 [==============================] - 18s 106us/step - loss: 5.4309 - _perplexity: 180.1502 - val_loss: 5.9389 - val__perplexity: 736.8768
Epoch 4/20
171249/171249 [==============================] - 18s 104us/step - loss: 5.2970 - _perplexity: 180.5992 - val_loss: 5.9496 - val__perplexity: 1322.4840
Epoch 5/20
171249/171249 [==============================] - 18s 105us/step - loss: 5.2141 - _perplexity: 183.7042 - val_loss: 5.9698 - val__perplexity: 2524.0300
Epoch 6/20
171249/171249 [==============================] - 18s 107us/step - loss: 5.1554 - _p

# Test Results

In [21]:
from src.dataset import load_tokenized_test
test_data = load_tokenized_test(tokenizer, with_melody=True)
first_words, song_embeddings = zip(*[(song['X'][0], song['melody_embedding']) for song in test_data])

def write_to_file(fname, lyrics):
    with open(fname, 'w+') as f:
        f.write(lyrics)

def parse_to_song(text):
    return text.replace('. ', '\n').replace(' \'', '\'').replace(' n\'t', 'n\'t').replace('eos', '')

def test_report(first_words):
    for i in range(5):
        curr = test_data[i]
        predicted_lyrics = model.predict(first_word=first_words[i], song=song_embeddings[i], n_words=250)
        lyrics = parse_to_song(predicted_lyrics)
        print(f"song_name: {curr['song_name']}\nartist: {curr['artist']}\nfirst word: {first_words[i]}\nPredicted song:\n{lyrics}\n\n")
        print("===========================================")
        song_name = curr['song_name'].rstrip().replace(' ', '_')
        fname=f'generated_lyrics\\lyrics_melody_doc2vec_model\\{song_name}\\{first_words[i]}.txt'
        write_to_file(fname, lyrics)

Loading the songs embedding: 100%|█████████████████████████████| 5/5 [00:01<00:00,  3.99it/s]


In [22]:
test_report(first_words)

song_name: eternal flame
artist: the bangles
first word: close
Predicted song:
close your beat of the roof you do 
care 
to see you just your line 
you into my heart's he's all trash 
now and them side 
oh yeah must a next again 
when you were true 
i'll saw keep going down again 
does let me 
 around 
down so much your waiting for you can flipside 
to walking around 
this problems i'm talkin jingle than there's mine 
since known high is ..you ow 
line 
ain't need me start to said 
the same 
you and they come saying you so way 
my knees 
for me sail down to you belong to see on the class with me into my eyes and he's sang me 
i king of preach you're amelia mocha only next let me 
charlie mony 
y'all show it's not it all of your boy 
you go to get his dich time before you edge of fighting for you're fast life 
rug ba dream are past and the mind 
broken baby give my before me that i playing midnight love was not the king 
what you're gon na make build me anyway as the other 
and the love

In [23]:
first_words = ['deep']*5
test_report(first_words)

song_name: eternal flame
artist: the bangles
first word: deep
Predicted song:
deep 
the bit 
i smiling before candle behind me 
between me like i'll you baby i never again 
and everyone darlin i can't garner 
when he's nothing to do it was social pretty world's fun 
and i've make heaven after outer ago 
to me caught cup 
sail ring my abroad and a place to dreams start to your chains girl 
you tight join ah the coat and the push of and see for the king to even take 
they by the water bring me what care 
foolish sein have to a often i will so sun's he's there who i know 

charlie 
you're walk fly runaway 
bout the bed i do what thought girl 
how called me a 
mistakes i do you're knowing these true 
we all my slowly 
when i joy above the church 
my heart can was on about it all the grounded 
follow you need a right and the hoo night think i do some high 
i'm mind's not it doesn't matter 
sure ends 
i was so long hard 
i'm young what 
you and me amadeus somehow me 
to forget tried to see m

In [24]:
first_words = ['hello']*5
test_report(first_words)

song_name: eternal flame
artist: the bangles
first word: hello
Predicted song:
hello oh brick and the dressed so now 
woo-hoo with a bottle of broadway as i only blowing through me you knew 
 he can feel so me 
the way 
and sabotage 
you're sails few 
no of was right the embrace 
but dreams or um oh oh and i've not why 
bailamos 
the middle is life 
but you'll never only sure i do you check's another tugboat lie's crazy egomaniac in so perfect allentown 
tell me come rock your tropic too 
no we're gon got a rules the little bit 
wonders of the nice chances 
you did i beckoning me belong 
i love me do to feel baby i'm telling me 
i don't get you make you i could never burn 
they askin is girl for you're something he corner 
and pum la la la twist before a city away 

and brighter girl what you 
oh its about a steam 
and all your place to fool 
no-one right 
whisper say 
'cause i village 
yeah dreams i see baby 
alone is more 
you don't cry 
'cause he apology your you're scared 

we said